In [3]:
import os
import sys
import random
import mne
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from pytorch_metric_learning import losses
from torchsummary import summary

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, normalize
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats import mode

import pandas as pd
import logging
import time
import math
import json
from collections import Counter

In [4]:
def get_names( ch_num, l ):
    name_list = []
    for i in range(ch_num):
        name_list.append( l[i]['name'][0] )
        # print(l[i]['name'][0])
    return name_list


def top_n_elements(lists, n=6):
    # 创建一个 Counter 对象以统计所有子列表中的元素
    overall_counter = Counter()
    
    # 遍历每个子列表并更新计数
    for sublist in lists:
        overall_counter.update(get_names(n, sublist))

    # 获取前 n 名元素
    top_elements_with_counts = overall_counter.most_common(n)

    top_elements = [element for element, count in top_elements_with_counts]
    counts = [count for element, count in top_elements_with_counts]
    
    return top_elements, counts

In [5]:
data_folder = 'E:\\EEG\\chb-mit-scalp-eeg-database-1.0.0'
COMMON_CH_list = []

for patient_index in range(1, 24+1):
    PATIENT_TO_USE = 'chb'+str(patient_index).zfill(2)
    with open(PATIENT_TO_USE+'_sel_ch_30iter_with_SMOTE.json', 'r') as json_file:
        loaded_data = json.load(json_file)
    COMMON_CH, _ = top_n_elements( loaded_data, 8 ) # 8 channels are acceptable
    COMMON_CH_list.append( set(COMMON_CH) )

In [6]:
common_channels = COMMON_CH_list[0]
for ch_set in COMMON_CH_list[1:]:
    common_channels = common_channels.intersection( ch_set )
    print( common_channels )

print( common_channels)

{'P4-O2', 'P8-O2', 'P3-O1', 'C4-P4'}
{'C4-P4', 'P4-O2', 'P3-O1'}
{'P4-O2', 'P3-O1', 'C4-P4'}
{'C4-P4'}
{'C4-P4'}
{'C4-P4'}
{'C4-P4'}
{'C4-P4'}
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [7]:
cnt = {}
for com_chs in COMMON_CH_list:
    for ch in list(com_chs):
        if ch not in cnt:
            cnt[ch] = 1
        else:
            cnt[ch] += 1

sorted( cnt.items(), key=lambda x:x[1], reverse=True)

[('P8-O2', 18),
 ('C3-P3', 18),
 ('P3-O1', 18),
 ('FZ-CZ', 17),
 ('C4-P4', 17),
 ('P4-O2', 17),
 ('CZ-PZ', 15),
 ('F3-C3', 14),
 ('P7-O1', 13),
 ('T7-P7', 10),
 ('F4-C4', 10),
 ('F7-T7', 6),
 ('FP2-F8', 5),
 ('F8-T8', 5),
 ('FP1-F3', 4),
 ('FP1-F7', 3),
 ('FP2-F4', 2)]

In [11]:
chs_for_ind = set(['P3-O1', 'P8-O2', 'C3-P3', 'C4-P4', 'FZ-CZ', 'P4-O2', 'CZ-PZ', 'F3-C3'])

for com_chs in COMMON_CH_list:
    print( chs_for_ind.intersection(com_chs) )

{'P8-O2', 'FZ-CZ', 'CZ-PZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'F3-C3', 'C4-P4', 'P3-O1', 'P4-O2'}
{'FZ-CZ', 'CZ-PZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P4-O2', 'P8-O2', 'P3-O1', 'C4-P4'}
{'F3-C3', 'FZ-CZ', 'C4-P4', 'CZ-PZ', 'C3-P3'}
{'F3-C3', 'C4-P4', 'CZ-PZ', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'FZ-CZ', 'C4-P4', 'CZ-PZ', 'C3-P3', 'P3-O1', 'P4-O2'}
{'F3-C3', 'FZ-CZ', 'C4-P4', 'CZ-PZ', 'C3-P3'}
{'P8-O2', 'F3-C3', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'F3-C3', 'FZ-CZ', 'CZ-PZ', 'P4-O2'}
{'P8-O2', 'F3-C3', 'FZ-CZ', 'CZ-PZ', 'P3-O1', 'P4-O2'}
{'P8-O2', 'P4-O2', 'P3-O1'}
{'P8-O2', 'FZ-CZ', 'CZ-PZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'FZ-CZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'F3-C3', 'FZ-CZ', 'C4-P4', 'CZ-PZ', 'C3-P3', 'P3-O1'}
{'P8-O2', 'F3-C3', 'FZ-CZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'FZ-CZ', 'CZ-PZ', 'C4-P4', 'C3-P3', 'P3-O1', 'P4-O2'}
{'P8-O2', 'F3-C3', 'FZ-CZ', 'C3-P3', 'P3-O1'}
{'P8-O2'}
{'C3-P3', 'F3-C3', 'FZ-CZ'}
{'P8-O2'

In [10]:
len( COMMON_CH_list )

24